# Bis zum High-Score! Wie ein Roboter durch Ausprobieren lernt.

## Storyboard 

<div style="text-align: justify">
    Eine der wichtigsten Aufgaben eines Roboters in der Automatisierung sind sogenannte <i>Pick and Place</i> Aufgaben. Dabei soll der Roboter ein Werkstück von einem Startpunkt zu einem Ziel bewegen und dort absetzen. Solche Aufgaben können relativ einfach gelöst werden, indem man dem Roboter die Bewegung mittels <i>Teach-in</i> beibringt. Beim Teach-in wird der Roboter händisch in wichtige Posen, wie z.B. die Aufhebe- und Absetzpose, gebracht und der Roboterzustand gespeichert. Bewegungen zwischen diesen Zuständen werden dann mittels klassischen Methoden der Robotik geplant und ebenfalls abgespeichert. So entsteht ein Bewegungsablauf, welcher abgespielt wird, um die Pick and Place Aufgabe durchzuführen. Der große Nachteil dieses Vorgehens ist, dass die Bewegung nicht dynamisch abgeändert werden kann. Kommen Variationen in Aufnahme- und Ablegeposen oder in den auszuweichenden Hindernissen vor, muss das Teach-in nochmals durchgeführt werden.
    <br /><br />
    
    Soll der Roboter nun z.B. ein Werkstück von einer sich immer leicht ändernden Position aufheben, geraten wir an die Grenzen des Teach-in. Hier ist es nicht wirtschaftlich, für jede mögliche Position des Werkstückes ein neues Teach-in durchzuführen. Die klassische Robotik bietet hier eine Lösung. Dabei beschreiben wir den Aufbau des Roboters mathematisch und ermitteln rechnerisch einen Pfad zwischen Start und Ziel. Auf Hindernisse im Arbeitsbereich Rücksicht zu nehmen verkompliziert jedoch so eine Implementierung. Deshalb stellen wir uns die Frage, ob wir mithilfe von AI den Roboter automatisch ein Verhaltensmuster zur Ausführung der Pick and Place Aufgabe lernen lassen können.
    </div>

### Reinforcement Learning

<div style="text-align: justify">
    Reinforcement Learning (RL, siehe AIAV Video <a href="https://www.youtube.com/watch?v=gNjDBkJRRP0&list=PLfJEPw9Zb0EPLEZZlNCQc9F3F7RWG6EsK&index=13">Reinforcement Learning)</a> beschäftigt sich mit dem Lernen durch intelligentes Ausprobieren. Dabei beschreiben wir das zu lösende Problem als Interaktion zwischen einem Agenten, z.B. einem Roboter, und dessen Umfeld (Environment). Der Agent befindet sich in einem Zustand (State) im Umfeld und führt eine der möglichen Aktionen (Action) aus. Das Ausführen der Aktion verändert den Zustand des Umfeldes und der neue, aus dem vorherigen Zustand und der Aktion resultierende, Zustand (new State) wird zurückgegeben. Zusätzlich zum neuen Zustand gibt das Umfeld auch Feedback darüber, wie erfolgreich die durchgeführte Aktion im Erreichen eines Ziels war. Dieses Feedback kommt in Form einer Zahl, dem Reward. Der Reward funktioniert wie die Punkteanzahl in einem Spiel - je höher, desto besser. Ein positiver Reward heißt also, dass die durchgeführte Aktion dem Agenten geholfen hat, sein Ziel zu erreichen. Analog dazu kennzeichnet ein negativer Reward das Scheitern des Agenten und ein Reward von 0 eine neutrale Aktion. Dieser Ablauf, der <a href="https://towardsdatascience.com/understanding-the-markov-decision-process-mdp-8f838510f150">Markov Decision Process (MDP)</a>, ist in Abbildung 1 dargestellt.
    </div>

<img src="images/Abbildung1MDP.jpg" alt="Drawing" style="width: 600px;"/>

_Abbildung 1: Aufgaben werden als Markov Decision Process (MDP) dargestellt, damit sie mittels Reinforcement Learning gelöst werden können. Dabei beschreibt der MDP eine Interaktion zwischen dem Lerner (Agenten) und seinem Umfeld (Environment)._

<div style="text-align: justify">
    Haben wir eine Aufgabe nun als MDP dargestellt, können wir uns überlegen wie der Agent überhaupt lernen soll. Wärhend viele verschiedene Algorithmen zum Lösen von Reinforcment Learning Problemen eingesetzt werden, haben diese alle das gleiche Ziel: den Reward mit der Zeit zu maximieren. Also soll unser Agent abhängig vom aktuellen Zustand Aktionen wählen, welche den höchsten Reward bringen. Am Anfang des Trainings weiß der Agent allerdings noch nichts über das Umfeld und die zu erfüllende Aufgabe. Deshalb werden zunächst nur zufällige Aktionen durchgeführt, um das Umfeld zu erkunden. Mit der Zeit basieren dann die gewählten Aktionen immer weniger auf Zufall und immer mehr auf der gelernten Dynamik der Umgebung.
    </div>

### Praktische Implementierung

<div style="text-align: justify">
    Um zu zeigen, wie ein praktisches Problem mittels Reinforcement Learning gelöst werden kann, wurde für diesen Usecase eine Pick and Place Aufgabe realisiert. Dabei werden Werkstücke durch einen Roboter mit drei Freiheitsgraden von einem Tisch aufgehoben. Das Werkstück kann an irgendeiner Position am Tisch, in Reichweite des Roboters, liegen. Zusätzlich dazu, gibt es noch Hindernisse im Arbeitsbereich des Roboters, die wir bei der Pfadplanung berücksichtigen müssen. Um Kollisionen mit Hindernissen zu vermeiden, wollen wir Reinforcement Learning verwenden, um den Pfad zwischen Start- und Zielpose zu planen.
    <br /><br />
    
    Aber wie formuliert man so ein vorliegendes Problem als Markov Decision Process?
    <br /><br />
    
    Zunächst legen wir fest, dass der Zustand (State) aus den Stellungen der drei Gelenke des Roboters besteht. Als mögliche Aktionen werden das Vergrößern oder Verkleinern der Stellung jedes Gelenks angenommen. Das heißt, dass der Roboter mit jedem Schritt eine von 6 Aktionen ausführen kann: Gelenk 1 erhöhen, Gelenk 1 vermindern, Gelenk 2 erhöhen, und so weiter. Da Zustand und Aktionen festgelegt sind, fehlt uns noch der Reward. Mit diesem signalisieren wir einerseits Erfolg und Versagen des Agenten, andererseits können wir aber auch, abhängig von der Komplexität der Reward Funktion, die Aufgabe schwerer oder einfacher für den Agenten gestalten. 
    <br /><br />
    
    Der Agent bekommt Reward anhand einer, als Teil der Umgebung definierten, Rewardfunktion. Für die Implementierung des Usecases haben wir uns für eine sogenannte spärliche Rewardfunktion entschieden. Das heißt, dass immer ein Reward von 0 signalisiert wird, außer der Agent hat sein Ziel erreicht oder ist gescheitert. Erreicht der Roboter also seine Zielpose, bekommt er einen Reward von 1, kollidiert er hingegen mit sich selbst oder Hindernissen, bekommt er einen Reward von -1 und ansonsten einen Reward von 0.
    <br /><br />
    
    Um nun konkrete Bahnen zu planen, legen wir die Start- und Zielposen fest. Dabei ist die Startpose der anfängliche Zustand des Agenten, während die Zielpose der einzige Zustand ist, der einen positiven Reward gibt. Sind Start und Ziel festgelegt, wenden wir <a href="https://towardsdatascience.com/policy-iteration-in-rl-an-illustration-6d58bdcb87a7">Policy Iteration</a>, ein Verfahren zum Lösen von Reinforcement Learning Problemen, an. Policy Iteration liefert uns als Lösung ein Verhaltensmuster (Policy). Dieses gibt für jeden Zustand an, welche der Aktionen in diesem ausgeführt werden müssen, damit positiver Reward generiert wird. Da positiver Reward nur vom Zielzustand kommt, entsteht durch die Aktionen eine Bahn zum Ziel. Abbildung 2 zeigt die Station sowie einen dieser geplanten Pfade. Dabei entspricht jeder Pfeil einem Eintrag des Verhaltensmusters und zeigt an, in welche Richtung sich der Greifer beim Ausführen der jeweiligen Aktion bewegt. 
    <br /><br />
    
    Unsere komplettes Vorgehen beim Formulieren des Reinforcement Learning Problems ist in diesem <a href="https://www.daaam.info/Downloads/Pdfs/proceedings/proceedings_2021/090.pdf">Paper</a> detaillierter beschrieben.
    </div>

<img src="images/Abbildung2StationMitPfad.jpg" alt="Drawing" style="width: 650px;"/>

_Abbildung 2: Durch Policy Iteration erhalten wir ein Verhaltensmuster, welchem der Roboter folgt um die Zielpose zu erreichen. Hier ist eines dieser Verhaltensmuster dargestellt. Dabei entspricht jeder Pfeil einem Zustand des Roboters, für den dieses Verhaltensmuster eine entsprechende Aktion vorgibt._

### Computer Vision

<div style="text-align: justify">
    Da wir nun Bewegungen automatisch mittels RL planen können, brauchen wir noch eine Möglichkeit, die Position des abgelegten Werkstücks zu bestimmen. Dafür montieren wir eine Kamera über dem Arbeitsbereich des Roboters und wenden klassische Bildverarbeitung an. Mittels dieser kann die Position des Werkstücks relativ zur Kamera bestimmt werden. Da die Position der Kamera vorab in der Station eingemessen wurde, haben wir einen Bezug zum Weltkoordinatensystem unserer Station und können die Position des Werkstücks durch Koordinatentransformation global nutzen. Die genaue Vorgehensweise bei der Objekterkennung ist in diesem <a href="https://www.daaam.info/Downloads/Pdfs/proceedings/proceedings_2021/077.pdf">Paper</a> beschrieben.
    <br /><br />
    
    Abbildung 3 zeigt die dabei angewendete Bildverarbeitungspipeline: Zunächst lesen wir das Bild ein und bestimmen einen Schwellenwert. Um das Werkstück vom hellen Hintergrund zu trennen, werden alle Bildpunkte über dem Schwellenwert auf weiß und alle Werte unter dem Schwellenwert auf schwarz gesetzt. Diesen Schritt nennt man Quantisierung. Anschließend <a href="https://docs.opencv.org/3.4.15/d7/de1/tutorial_js_canny.html">erkennen wir Kanten im Bild</a>, um die Kontur des Werkstücks zu ermitteln. Um die Kontur wird eine <a href="https://docs.opencv.org/3.4/da/d0c/tutorial_bounding_rects_circles.html">Box</a> gelegt. Die Mitte dieser Box ist dann die Position des Werkstücks im Bild. Da diese Position noch in Pixeln angegeben ist, muss sie in Meter umgerechnet werden. Diese Umrechnung basiert auf physikalischen Parametern der Kamera, wie die Größe des Sensors und der Brennweite der Linse. Diese Parameter wurden vorab durch Kalibrierung der Kamera ermittelt und abgespeichert.
    </div>

<img src="images/Abbildung3Pipeline.jpg" alt="Drawing" style="width: 750px;"/>

_Abbildung 3: Um die Position des Werkstücks zu ermitteln, wenden wir klassische Bildverarbeitung an. Dabei quantisieren wir zunächst das Kamerabild und erkennen die Kanten. Anschließend ziehen wir eine Box rund um das Werkstück und berechnen aus dessen Position im Bild, die Position des Werkstücks in der echten Welt._

### Fazit

<div style="text-align: justify">
    Reinforcement Learning erlaubt es uns, Planungsprobleme vorab in Simulation zu lösen und die Lösung auf die echte Welt anzuwenden. Neben der Auswahl des Algorithmus, mit dem wir den Agenten implementieren und das Problem lösen, müssen wir uns auch darüber Gedanken machen, wie wir das Problem überhaupt formulieren. Festzulegen, wie Zustand, Aktionen und Reward aussehen, ist ein wesentlicher Schritt im Lösen eines Problems mittels Reinforcement Learning.
    <br /><br />
    
    Die vorgestellte Lösung erlaubt es uns den Pfad automatisiert zu planen, sodass der Roboter die Pick and Place Aufgabe eigenständig lösen kann. Jedoch hat die Implementierung einige Grenzen. Dadurch, dass wir Policy Iteration zur Lösung des Reinforcement Learning Problems verwenden, muss unser Markov Decision Process komplett beobachtbar sein. Dafür müssen wir in der Simulation zunächst alle möglichen Zustände des Roboters abfahren und speichern. Das resultiert in einer Startzeit des Roboters von mehreren Minuten. Diese Limitierung könnte mit einer anderen Reinforcement Learning Methode, wie z.B. Q-Learning, umgangen werden.
    <br /><br />
    
    Dadurch, dass zur Erkennung des Werkstücks klassische farbbasierte Bildverarbeitung eingesetzt wird, ist diese aber bei sich ändernden Lichtverhältnissen fehleranfällig. Hier könnte der Einsatz moderner Methoden zur Objekterkennung, wie z.B. ein Convolutional Neural Network (CNN, siehe AIAV Video <a href="https://www.youtube.com/watch?v=xIXGDepixcg&list=PLfJEPw9Zb0EPLEZZlNCQc9F3F7RWG6EsK&index=5">CNN Detector</a>) die Fehleranfälligkeit der Lösung verbessern.
    </div>
    
<P style="page-break-before: always">

## Code Dokumentation

Traditionelle Pfadplanungsverfahren benötigen typischerweise Wissen über die physikalischen Einschränkungen des Roboters, um einen effizienten Pfad zum Ziel zu planen [[1]](https://doi.org/10.1007/BFb0036074). Dabei stellt sich die Frage, ob künstliche Intelligenz verwendet werden kann, um ein System zu implementieren, welches die physikalischen Einschränkungen des zu steuernden Roboters eigenständig lernt.

Im Zuge dieser Arbeit wurde eine Pick and Place Aufgabe mittels Reinforcement Learning zur Pfadplanung realisiert. Ein Werkstück wird auf einer Ablagefläche abgelegt und soll von einem Dreiachsroboter in eine Computergesteuerte Fräse gelegt werden. Die Position, an welcher das Werkstück abgelegt wird, sowie das Umfeld des Roboters sind dabei variabel. Damit kann kein Ansatz mit statischer Pfadplanung verwendet werden. Damit der Roboter das Werkstück automatisiert aufheben kann, muss seine Position erkannt werden. Diese Erkennung wird mittels farbbasierter Computer Vision umgesetzt. Wir haben uns für ein Vorgehen basierend auf künstlicher Intelligenz entschlossen, da die Ablageposition variabel ist und da sich potentielle Hindernisse um den Greifroboter ändern können. Der Roboter muss also in der Lage sein die Position des Werkstücks zu erkennen und einen Pfad von der Aufnahme- zur Ablageposition zu planen und dabei nicht mit Hindernissen zu kollidieren. 

Die Pick and Place Aufgabe wurde mittels Reinforcement Learning (RL) zur Pfadplanung und farbbasierter Computer Vision zur Objekterkennung realisiert. Nach Ermittlung der Objektposition wird die Bahn des Roboters in der Simulation durch Policy Iteration, einem RL Verfahren, geplant. Die geplante Bahn wird in der Simulation getestet und am realen Roboter ausgeführt. Dabei wird ein Verhaltensmuster pro Zielpunkt traininert, welches dem Roboter erlaubt, den Zielpunkt von jeder gültigen Startpose aus zu erreichen.

Die Idee dieser Lösung ist es, ein System zu schaffen, welches auf keine exakte Ablageposition des Werkstücks angewiesen ist. Solange das Werkstück im Sichtfeld der Kamera sowie in Reichweite des Roboters abgelegt wird, kann es von der Station weiter Verarbeitet werden.

Benötigte Software Pakete werden zunächst importiert. Neben bestehenden Bibliotheken wie [numpy](https://numpy.org/), [ROS](https://www.ros.org/) und [OpenAI Gym](https://gym.openai.com/), werden auch für den Usecase implementierte Module geladen. *gym_fhtw3dof* startet eine Simulation des Roboters sowie seines Umfelds und erstellt ein Reinforcement Learning Environment auf Basis der OpenAI Gym API, in welchem die Pfadplanung vorgenommen wird. Das *hebi_publisher* Modul steuert die Motoren des Roboters. *fhtw3dof_usecase_tools* ist eine Implementierung der Policy Iteration, Value Iteration und Q-Learning Algorithmen basierend auf [Sutton & Barto's Reinforcement learning: An introduction](https://scholar.google.de/scholar?hl=de&as_sdt=0%2C5&q=sutton+barto+reinforcement+learning&btnG=&oq=sutton+bart), sowie [dieser](https://github.com/GiacomoFerro/Q_Learning_Games_v3) und [dieser](https://medium.com/analytics-vidhya/solving-the-frozenlake-environment-from-openai-gym-using-value-iteration-5a078dffe438) Implementierung. *incaccuracy_compensation* verwendet inverse Kinematik, um durch die diskrete Darstellung des Roboters auftetende Ungenauigkeiten auszugleichen.

In [ ]:
import numpy as np                    # Import von bereits bestehenden, verwendeten Python Bibliotheken
import time
import math

import gym                            # Import von Openai Gym
import gym_fhtw3dof                   # Import des Gym Environments, welches für den Usecase entwickelt wurde

import rospy                          # Import der Komponenten des Robot Operating Systems (ROS)
import tf
from geometry_msgs.msg import Point, PointStamped
from std_msgs.msg import Bool

import hebi_publisher                 # Das hebi_publisher Modul kommuniziert mit den Motoren des realen Roboters
import fhtw3dof_usecase_tools         # Das fhtw3dof_usecase_tools Modul berechnet die gewünsche Verhaltensweise des Roboters mittels Policy Iteration
import inaccuracy_compensation        # Das inaccuracy_compensation Modul verwendet inverse Kinematik, um durch das Environment entstehende Ungenauigkeiten auszugleichen

real = True                           # Einstellung, ob der Usecase nur simuliert, oder am echten Roboter ausgeführt wird

place_state = 34721                   # Die codierten Zustände im Environment, welche die Wartepose (Home) und die Ablagepose (Place) beschreiben
home_state = 29659

goalstate_distance = 0.04             # Distanz zum Zielpunkt, in dem nach Zielposen gesucht wird
grasp_height = -0.01                  # Z-Höhe, in der der Roboter versucht das Objekt zu greifen
goalstate_height = 0.04               # Z-Höhe, in der der an das Environment gegebene Zielzustand liegt

save_model = True                     # Einstellung, ob trainierte Modelle gespeichert werden sollen
load_pretrained = True                # Einstellung, ob bereits trainierte Modelle geladen werden sollen

### Hilfsfunktionen

Die Funktion *find_nearest_state()* sucht nach Zuständen des Environments, in welchen der TCP des Roboters innerhalb von *goalstate_distance* rund um den spezifizierten Zielpunkt liegt. Von diesen Zuständen wird der mit der kleinsten Distanz zum Zielpunkt *point* in kodierter Form zurückgegeben.

*get_object_position()*, lädt die Transformation vom Welt- zum Objektkoordinatensystem des erkannten Objektes. Diese Transformation wird vom Computer Vision System gesendet und dient zur Ermittlung der Objektposition relativ zum Roboter.

*compute_goal()* ermittelt den Zielzustand im Environment sowie die inverse Kinematik der Roboters im Zielzustand mittels des *inaccuracy_compensation* Moduls.

*goto_jointstate()* bewegt den Roboter zu einer Zielposition.

*apply_policy* führt ein berechnetes Verhaltensmuster am realen und simulierten Roboter aus. Dafür wird der Anfangszustand des Roboters gespeichert und aus diesem heraus Aktionen anhand des Verhaltensmusters getätigt, was wiederum in einem neuen Zustand resultiert. Dies wird solange durchgeführt, bis eine maximale Anzahl an Schritten erreicht ist, oder das Ziel erreicht wurde. Am realen Roboter wird nur jeder zweite Zustand ausgeführt, um seine Bahn zu glätten.

In [ ]:
def find_nearest_state(point, max_distance, env):
    """ Gibt den kodierten Zustand mit kleinster Distanz zu point zurück. """
    if (not isinstance(point, Point)): return -1          # Terminiert mit Fehlercode, wenn kein Punkt übergeben wurde
    
    states = env.find_near_states(point, max_distance)    # In Frage kommende Zustände werden aus dem Environment geladen
    if len(states) == 1: return states[0]                 # Falls nur ein Zustand innerhalb von max_distance gefunden wurde, wird dieser zurückgegeben

    goalstate = -1
    for s in states:                                      # Es wird über gefundene Zustände iteriert, und für jeden die Distanz berechnet
        distance =  round(math.sqrt(math.pow(point.x - env.TCP_Positions[s].x, 2) + 
                                    math.pow(point.y - env.TCP_Positions[s].y, 2)), 6)
        try:
            if distance < max_distance:
                max_distance = distance
                goalstate = s
        except ValueError:
            continue
    return goalstate                                      # Der Zustand mit der kleinsten Distanz zu point wird zurückgegeben

def get_object_position():
    """ Ermittelt die Position des erkannten Objektes anhand der Transformation von Welt- zu Objektkoordinatensystem. """
    pt = PointStamped()                                   # Der Punkt pt wird an Position (0,0,0) im Weltkoordinatensystem instanziert
    pt.header.frame_id = "object_tf"
    pt.header.stamp = rospy.Time()
    pt.point.x = 0
    pt.point.y = 0
    pt.point.z = 0
    try:
        pt = tflistener.transformPoint("world", pt)      # pt wird in das Objektkoordinatensystem transformiert, um die Position des Objekts zu erhalten
    except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException):
        print("No Transform available")
        return -1
    return pt

def compute_goal(env, goalstate_distance):
    """ Ermittelt den Zustand, mit welchem der Roboter das erkannte Objekt erreicht und die dazugehörige inverse Kinematik. """
    pt = get_object_position()                           # Die Position des Objekts wird ermittelt
    if pt == -1: return -1, 0, 0, 0
    try:                                                 # Es wird versucht die inverse Kinematik zu lösen
        j3, j2, j1 = inaccuracy_compensation.getJointstate(pt.point.x, pt.point.y, grasp_height)
    except ValueError:
        print("Could not solve inverse kinematics")
        return -1, 0, 0, 0                               # Wird keine Lösung gefunden, wird ein Fehlercode zurückgegeben
    pt.point.z = goalstate_height                        # Der näheste Punkt zum Anfahren des Ziels auf angegebener Höhe wird ermittelt
    goalstate = find_nearest_state(pt.point, goalstate_distance, env)
    return goalstate, j3, j2, j1

def goto_jointstate(j3, j2, j1, env):
    """ Bewegt den Roboter zu dem angegebenen Zustand. """
    env.js.position[0] = j1                              # Die Winkelstellungen des simulierten Roboters werden gesetzt
    env.js.position[1] = j2
    env.js.position[2] = j3
    env.render()                                         # Der simulierte Roboter führt die Bewegung aus
    if real: hebi_pub.add_waypoint()                     # Die Bewegung des realen Roboters zu dieser Pose wird geplant

def apply_policy(policy, target_state, env):
    """ Führt ein erlerntes Verhaltensmuster am simulierten und realen Roboter aus. """
    if env.s == target_state: return 1                   # Prüfung ob der Zielpunkt schon erreicht ist
    obs = env.s                                          # Der Anfangszustand des Roboters wird gespeichert
    step_idx = 0
    while True:
        action = int(policy[obs])                        # Die zu tätigende Aktion wird anhand des Verhaltensmusters (policy) bestimmt
        obs, _, _, _ = env.step(action)                  # Die Aktion wird ausgeführt
        step_idx += 1
        env.render_state(obs)                            # Der resultierende Zustand wird am simulierten und virtuellen Roboter angewandt
        if real and (int(step_idx%2)==1): hebi_pub.add_waypoint() # Nur jeder 2. Zustand wird am realen Roboter ausgeführt um die Bahn zu glätten
        if obs == target_state:                          # Funktion wird beendet, wenn das Ziel erreicht ist, oder mehr als 100 Aktionen benötigt wurden
            return 1
        if step_idx >= 100: return -1
    return -1

### Programmablauf

Zunächst wird das Gym Environment mit Parametern für den im Usecase verwendeten Roboter als *env* initialisiert. Das Environment erstellt eine Simulation des Usecases in ROS und verwendet diese zum Training der Verhaltensweisen um zu einem Zielpunkt zu gelangen.

Die definierten Argumente werden vom *gym.make()* Befehl an den Konstruktor des Environments übergeben. Eine genaue Beschreibung der Argumente ist in der technischen Dokumentation des FHTW3DOF Paketes zur Verfügung gestellt.

In [ ]:
def setup_environment():
    """ Konfiguriert das fhtw3dof Gym Environment für die Nutzung mit dem SAImon Roboter anhand des Usecases. """
    env = gym.make('fhtw3dof-v0', Stop_Early=False, Constrain_Workspace=True,
                    GoalX=0.179, GoalY=0.0026, GoalZ=0.335,
                    J1Limit=33, J2Limit=35, J3Limit=37, joint_res = 0.08,
                    J1Scale=1, J2Scale=-1, J3Scale=-1, J3Offset=0.5,
                    Debug=True, Publish_Frequency=500)
    env.Publish_Frequency = 20                        # Die Bewegungsgeschwindigkeit des virtuellen Roboters wird für eine übersichtlichere Visualisierung verlangsamt
    return env

rospy.init_node("aiav")                               # Initialisierung einer ROS Node, zur Kommunikation mit den Roboter-Spezifischen Komponenten

env = setup_environment()                             # Initialiserung des Gym Environments
env.s = home_state                                    # Der virtuelle Roboter wird in seinen Wartezustand gesetzt
env.render_state(env.s)

*TransformListener* und *iterations* Objekte werden Instanziert, um die Objektposition zu ermitteln und um Verhaltensmuster anhand von *env* zu berechnen.

In [ ]:
policy_iteration = fhtw3dof_usecase_tools.iterations() # policy_iteration Objekt berechnet die Verhaltensmuster mittels Policy Iteration

tflistener = tf.TransformListener()                    # tflistener wird zum Laden er Objektposition benötigt

Werden bereits trainierte Modelle verwendet, werden die entsprechenden Dateien geladen. Das Modell besteht aus bereits bekannten Zielzuständen und zugehörigen Verhaltensmustern. Verhaltensmuster zum erreichen des Warte- und Ablegzustandes werden in extra Dateien gespeichert.

Falls keine vortrainierten Modelle verwendet werden (z.B. nach Änderung des Roboters) werden Verhaltensmuster zum erreichen des Ziel- und Wartezustandes wärhend des Initialisierungsvorgangs trainiert und gespeichert.

In [ ]:
if load_pretrained:                                        # Werden vortrainierte Modelle verwendet, werden die entsprechenden Dateien geladen
    known_goalstates = np.loadtxt('known_goalstates.txt', dtype=int).tolist()
    policies = np.loadtxt('policies.txt', dtype=int).tolist()
    place_policy = np.loadtxt('place_policy.txt', dtype=int).tolist()
    home_policy = np.loadtxt('home_policy.txt', dtype=int).tolist()
else:                                                      # Ansonsten werden Ziel- und Wartezustand trainiert
    known_goalstates = []
    policies = []
    
    # Berechnung des Verhaltensmusters zum Erreichen des Ablegzustands
    env.P = env.reset_rewards(env.P, 0, env.nS)            # Alle positiven Rewards im Environment werden zurückgesetzt
    env.P, idx = env.set_state_reward(env.P, place_state)  # Reward für das Erreichen des zuz trainierenden Zustandes wird gesetzt
    if idx < 1:                                            # Eine Fehlermeldung wird ausgegeben, falls der Zustand nicht erreichbar ist
                print("Place state not reachable.")
                quit()
    place_policy = policy_iteration.policy_iteration(env)
    
    # Berechnung des Verhaltensmusters zum Erreichen des Wartezustands
    env.P = env.reset_rewards(env.P, 0, env.nS)
    env.P, idx = env.set_state_reward(env.P, home_state)
    if idx < 1:
                print("Home state not reachable.")
                quit()
    home_policy = policy_iteration.policy_iteration(env)


Wird der Usecase am echten Roboter ausgeführt, wird dessen Ansteuerung gestartet. Das *hebi_publisher* Modul synchronisiert den virtuellen und realen Roboter und erlaubt die Planung einer Bewegung anhand in der Simulation ausgeführter Bewegungen.

In [ ]:
if real: hebi_pub = hebi_publisher.hebi_publisher()

Der Ablauf der Usecase Steuerung erfolgt anhand folgender Schleife:

1. Warte auf Eingabe durch Aufsichtsperson
2. Ermittlung von Zielpunkt und inverser Kinematik um diesen zu erreichen
3. Prüfung, ob der Zielpunkt erreichbar ist
  * Wenn nein, dann Ausgabe einer Fehlermeldung und Neustart der Schleife
  * Wenn ja, dann wird die Schleife weiter ausgeführt
4. Prüfung, ob der Zielpunkt bereits bekannt ist
  * Wenn nein, wird ein Verhaltensmuster zum erreichen des Ziels trainiert und gespeichert
  * Wenn ja, dann wird das entsprechende Verhaltensmuster geladen
5. Bewegung anhand des Verhaltensmusters wird geplant
6. Extra Punkt, resultierend aus der inversen Kinematik, wird der Bewegung hinzugefügt
7. Bewegung wird ausgeführt
7. Greifer wird geschlossen
8. Bewegung zum Absetzpunkt wird geplant und ausgeführt
9. Greifer wird geöffnet
10. Bewegung zum Wartepunkt wird geplant und ausgeführt

Bei der Planung der Bewegung, wird der von dem Verhaltensmuster geplanten Bahn noch ein Punkt, resultierend aus den Winkelstellungen aus der inversen Kinematik, hinzugefügt, um die Genauigkeit des Roboters zu erhöhen.

Ist ein anzufahrender Zielzustand noch nicht bekannt, werden zunächst positive Rewards im Environment auf null gesetzt. Damit bleiben Informationen über Hindernisse, bei welchen der Reward -1 ist, erhalten, während Ziele zurückgesetzt werden. Das Reward-Signal für den Zielzustand wird dann auf 1 gesetzt; das Ziel des Algorithmus ist es, diesen Zustand zu erreichen. Das Verhaltensmuster zum Erreichen des Zielzustands wird anhand von Policy Iteration durch das Modul *fhtw3dof_usecase_tools* ermittelt und zu den bekannten Modellen hinzugefügt. Wenn ein Zielzustand bekannt ist, kann dieser durch das trainierte Verhaltensmuster von jedem validen Startzustand aus erreicht werden.

In [ ]:
while True:
    try:
        int(input())                                             # (1) Start des Schleifendurchlaufes, wird etwas anderes als ein Integer eingegeben, wird der Ablauf beendet
    except ValueError:
        break    
        
    goal_state, j3_goal, j2_goal, j1_goal = compute_goal(env, goalstate_distance) #  (2 & 3) Zielzustand wird ermittelt und auf Validität überprüft
    if goal_state < 0 or goal_state >= env.nS:
        print("Goalstate is not within the Robot's State-Size")
        continue
    if env.Colliding_States[goal_state]:
        print("Goalstate is Colliding with Cbstacles.")
        continue
    
    try:                                                         # (4) Falls der Zielzustand schon bekannt ist, wird der Speicherort des entsprechenden Modells ermittelt
        idx = known_goalstates.index(goal_state)
    except ValueError as err:
        
        env.P = env.reset_rewards(env.P, 0, env.nS)              # Environment wird zurückgesetzt
        env.P, idx = env.set_state_reward(env.P, goal_state)     # Reward-Signal wird für den Zielzustand auf 1 gesetzt
        if idx < 1:                                              # Überprüfung, ob der Zielzustand trainiert werden kann
            print("Goalstate not reachable.")
            continue
        
        tmp_policy = policy_iteration.policy_iteration(env)      # Berechnung des Verhaltensmusters zum Erreichen des Zielzustandes
        
        known_goalstates.append(goal_state)                      # Das trainierte Modell wird zur Wiederverwendung gespeichert
        policies.append(tmp_policy)
        idx = len(known_goalstates)-1
    
    r  = apply_policy(policies[idx], goal_state, env)            # (5) Ausführung des Verhaltensmuster am simulierten Roboter
    
    goto_jointstate(j3_goal, j2_goal, j1_goal, env)              # (6) Anfahren des Punktes, der durch die inverse Kinematik bestimmt wird
    
    if real: hebi_pub.publish_trajectory_to_robot()              # (7) Ausführen der virtuell ausgeführten Bahn am realen Roboter
    time.sleep(8)
    
    print("Closing gripper")                                     # (8) Greifer wird geschlossen
    if real: hebi_pub.close_gripper()
    time.sleep(5)
    
    r = apply_policy(place_policy, place_state, env)             # (9) Bewegung zum Absetzpunkt wird geplant und ausgeführt
    if real: hebi_pub.publish_trajectory_to_robot()
    time.sleep(8)
    
    print("Opening gripper")                                     # (10) Greifer wird geschlossen
    if real: hebi_pub.open_gripper()
    time.sleep(5)
    
    r = apply_policy(home_policy, home_state, env)               # (11) Bewegung zum Wartepunkt wird geplant und ausgeführt
    if real: hebi_pub.publish_trajectory_to_robot()
    time.sleep(8)


Ist die entsprechende Variable gesetzt, werden die trainierten Modelle zum Laden beim nächsten Programmdurchlauf gespeichert.

In [ ]:
if save_model:                                                  # Trainierte Modelle werden zur späteren Verwendung gespeichert
    np.savetxt('home_policy.txt', home_policy,fmt='%d')
    np.savetxt('place_policy.txt', place_policy,fmt='%d')
    np.savetxt('policies.txt', policies,fmt='%d')
    np.savetxt('known_goalstates.txt', known_goalstates,fmt='%d')
    print("Policies saved successfully.")